In [44]:
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import time

#DB 작업
import pymysql

In [45]:
# 파일 로드
def load_file(fileName):
    return pd.read_csv(f'data/{fileName}', encoding='cp949', index_col=0)

In [46]:
# place_name 갖고오기
def get_key(amuse):
    keyword_list=[]
    for keyword in amuse['place_name']:
        #print(keyword)
        keyword = keyword.replace(" ", "+")
        keyword_list.append(keyword)
    return keyword_list

In [48]:
def text_cleaning(text) :
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

In [49]:
def filtering_keyword(titleList):
            
# 한글만 추출하기
    titleList = list(map(lambda x:text_cleaning(x), titleList))
    #print(titleList)
    
    title_corpus = " ".join(titleList) # 추출한 각 요소를 띄어쓰기로 join
    nouns_tagger = Okt() # 형태소 초기화
    nouns = nouns_tagger.nouns(title_corpus) # 명사 추출
    
    count = Counter(nouns) # 명사 갯수 세기

    #print(count)
    
    remove_char_counter = Counter({x:count[x] for x in count if len(x) > 1})    # 데이터 길이가 1보다 큰 것만 필터링
    #print(remove_char_counter)
    
    # 불용어 처리 경로 지정
    korean_stopwords_path = "data/stopwords-ko.txt"
    
    # stopwords 파일 열기
    with open(korean_stopwords_path, encoding='utf8') as f:
        stopwords = f.readlines()
    stopwords = [x.strip() for x in stopwords]
    
     # 위에서 필터링한 리스트를 하나씩 for 문 돌려서 stopwords에 해당되지 않는 것만 갯수 세기
    remove_char_counter = Counter({x:remove_char_counter[x] for x in remove_char_counter if x not in stopwords})
    #print(remove_char_counter)

    return remove_char_counter
    

In [50]:

def trans_dict(remove_char_counter):

    # 빈도수 오름차순으로 50개 가져오기
    ranked_tags = remove_char_counter.most_common(50) # 빈도순 n개 단어
    # 딕셔너리로 변환
    ranked_tags = dict(ranked_tags)
    # 단어와 빈도를 dataframe으로 갖고오기
    df = pd.DataFrame(ranked_tags.items(), columns=['단어', '빈도'])
    
    return df

In [59]:
rawData = url_raw(get_key(load_file('pj1_amusementpark_01.csv')))

print(trans_dict(filtering_keyword(rawData)))






https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=1
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=2
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=3
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=4
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=5
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=6
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=7
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=8
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=9
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=


# DB 작업 시작

In [53]:
'''
# MariaDB 연결
conn = pymysql.connect(host='192.168.0.103', user='user1', password='u1234', db='dbmaria', charset='utf8')
# 커서 생성
cur = conn.cursor()
'''




       단어  빈도
0      천안  35
1      노벨  24
2      창업  14
3      캠프  11
4      충남  10
5      프레   9
6      웨이   9
7      강원   8
8      리솜   8
9      컬처   8
10     파크   7
11    박람회   7
12  독립기념관   7
13   셔틀버스   7
14     시즌   6
15     세션   6
16     성장   6
17     식음   6
18    서비스   6
19   오션월드   6
20     개최   6
21    리조트   6
22     학생   6
23    브랜드   5
24    스토어   5
25     보증   5
26     일간   5
27     임시   5
28    주차장   5
29     운행   5
30    대전대   5
31     학년   5
32     호텔   4
33     비치   4
34     서울   4
35     홍천   4
36     스플   4
37     라스   4
38     예산   4
39     웰리   4
40     힐리   4
41     횡성   4
42   포레스트   4
43     충북   4
44     팩트   4
45     농협   4
46     코스   4
47     주차   4
48     편의   4
49     위해   4


In [ ]:
for keyword in keyword_list[:3]:
    
    titleList = []
    
    # 페이지 수 지정하여 for문 실행
    for i in range(1, 4, 1):
        url = f"https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={i}"
        print(url)
        response = requests.get(url) # url 갖고오기
        html = response.text # text만 갖고오기
        soup = BeautifulSoup(html, 'html.parser') # 파싱하기
        titles = soup.select("ul.c-list-basic") # c-list-basic 클래스에서 ul에 묶여있는 요소들을 모두 갖고온다

        #subtitles = soup.select("a.elss.sub_tit")
        for title in titles : # titles에서 title   # 갖고온 요소로 for문 실행
            titleText = title.text # title.text만 추출
            titleList.append(titleText) # titleList에 추가
        
        time.sleep(5)
    
    print(trans_dict(filtering_keyword(titleList)))
        
    print("-----구분선-----")

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=1
https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=이월드&p=2
